In [1]:
import numpy as np
import pandas as pd

In [2]:
file=r'C:\Users\akhil\Downloads\Data\Data\Existing Base.csv'

In [3]:
data=pd.read_csv(file)

In [4]:
data.head()

,REF_NO,children,age_band,status,occupation,occupation_partner,home_status,family_income,self_employed,self_employed_partner,...,Investment Tax Saving Bond,Home Loan,Online Purchase Amount,Revenue Grid,gender,region,Investment in Commudity,Investment in Equity,Investment in Derivative,Portfolio Balance
0,1,Zero,51-55,Partner,Manual Worker,Secretarial/Admin,Own Home,"<17,500, >=15,000",No,No,...,19.99,0.00,0.00,1,Female,Wales,74.67,18.66,32.32,89.43
1,2,Zero,55-60,Single/Never Married,Retired,Retired,Own Home,"<27,500, >=25,000",No,No,...,0.00,0.00,0.00,2,Female,North West,20.19,0.00,4.33,22.78
2,3,Zero,26-30,Single/Never Married,Professional,Other,Own Home,"<30,000, >=27,500",Yes,No,...,0.00,3.49,0.00,2,Male,North,98.06,31.07,80.96,171.78
3,5,Zero,18-21,Single/Never Married,Professional,Manual Worker,Own Home,"<15,000, >=12,500",No,No,...,0.00,0.00,0.00,2,Female,West Midlands,4.10,14.15,17.57,-41.70
4,6,Zero,45-50,Partner,Business Manager,Unknown,Own Home,"<30,000, >=27,500",No,No,...,0.00,45.91,25.98,2,Female,Scotland,70.16,55.86,80.44,235.02


In [6]:
data.isnull().sum()
# there are no missing values in data if exists we would have made it as a sub-pipeline

REF_NO                             0
children                           0
age_band                           0
status                             0
occupation                         0
occupation_partner                 0
home_status                        0
family_income                      0
self_employed                      0
self_employed_partner              0
year_last_moved                    0
TVarea                             0
post_code                          0
post_area                          0
Average Credit Card Transaction    0
Balance Transfer                   0
Term Deposit                       0
Life Insurance                     0
Medical Insurance                  0
Average A/C Balance                0
Personal Loan                      0
Investment in Mutual Fund          0
Investment Tax Saving Bond         0
Home Loan                          0
Online Purchase Amount             0
Revenue Grid                       0
gender                             0
r

In [15]:
#Now we split data to train and test
from sklearn.model_selection import train_test_split
bd_train,bd_test=train_test_split(data,test_size=0.2,random_state=2)


In [14]:
x_train=bd_train.drop('Revenue Grid',axis=1)
y_train=bd_train['Revenue Grid']
x_test=bd_test.drop('Revenue Grid',axis=1)
y_test=bd_test['Revenue Grid']

In [19]:
from sklearn.base import BaseEstimator,TransformerMixin

In [26]:
class VarTypeSelector(BaseEstimator,TransformerMixin):
    
    def __init__(self,vartype,ignore_var):
        self.vartype=vartype
        self.ignore_var=ignore_var
        
    def fit(self,x,y=None):
        return self
    
    def transfrom(self,x,y=None):
        return x.select_dtypes(self.vartype).drop(slef.ignore_var,axis=1)

In [27]:
class get_dummies(BaseEstimator,TransformerMixin):
    
    def __init__(self,freq=0):
        self.freq=freq
        self.cat_dict={}
        
    def fit(self,x,y=None):
        data_cols=x.columns
        for col in data_cols:
            k=x[col].value_counts()
            cats=k.index[k>slef.freq][:-1]
            self.cat_dict[col]=cats
        return self
    
    def transform(self,x,y=None):
        dummy_data=x.copy()
        for col in self.cat_dict.keys():
            for cat in self.cat_dict[col]:
                name=col+'_'+cat
                dummy_data[name]=(dummy_data[col]==cat).astype(int)
            del dummy_data[col]
        return dummy_data
        

In [22]:
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.linear_model import LogisticRegression

In [36]:
class VarTypeSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self,vartype,ignore_var):
        self.vartype=vartype
        self.ignore_var=ignore_var
    
    def fit(self,x,y=None):
        return self
    
    def transform(self,X):
        return X.select_dtypes(self.vartype).drop(self.ignore_var,axis=1)

class get_dummies_PipeLineFriendly(BaseEstimator, TransformerMixin):
    
    def __init__(self,freq_cutoff=0):
        self.freq_cutoff=freq_cutoff
        self.var_cat_dict={}
        
    def fit(self,x,y=None):
        data_cols=x.columns
        for col in data_cols:
            k=x[col].value_counts()
            cats=k.index[k>self.freq_cutoff][:-1]
            self.var_cat_dict[col]=cats
        return self
            
    def transform(self,x,y=None):
        dummy_data=x.copy()
        for col in self.var_cat_dict.keys():
            for cat in self.var_cat_dict[col]:
                name=col+'_'+cat
                dummy_data[name]=(dummy_data[col]==cat).astype(int)
            del dummy_data[col]
        return dummy_data

In [37]:
cat_pipe=Pipeline([
    ('cat_var',VarTypeSelector(['object'],ignore_var=['post_code','post_area'])),
    ('dummies',get_dummies_PipeLineFriendly(100))
])

In [38]:
pipe2=Pipeline([
    ('features',FeatureUnion([
        ('cat_pipe',cat_pipe),
        ('num_var',VarTypeSelector(['int64','float64'],ignore_var=['REF_NO']))
    ])),
    ('clf',LogisticRegression())
])

In [39]:
pipe2.fit(x_train,y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('cat_pipe', Pipeline(memory=None,
     steps=[('cat_var', VarTypeSelector(ignore_var=['post_code', 'post_area'], vartype=['object'])), ('dummies', get_dummies_PipeLineFriendly(freq_cutoff=100))])), ('num_var', VarTypeSelector(ignor...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [40]:
pipe2.predict_proba(x_test)

array([[0.00352334, 0.99647666],
       [0.00623159, 0.99376841],
       [0.00768033, 0.99231967],
       ...,
       [0.43372072, 0.56627928],
       [0.17600561, 0.82399439],
       [0.01433096, 0.98566904]])

In [41]:
pipe2.classes_

array([1, 2], dtype=int64)